In [10]:
import psutil
import urllib.request
import json
from PyPDF2 import PdfReader

In [1]:
import psutil

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [7]:
import urllib
import json
def query_model(
    prompt,
    model="llama3",
    url="http://localhost:11434/api/chat"
):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }


    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

In [11]:
def upload_and_convert_pdf(pdf_path):
    """Convert a PDF document into plain text."""
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [13]:
def evaluate_responses(text, models, evaluator_model="llama3"):
    """Summarize text using multiple models and evaluate their performance."""
    # Get summaries from smaller models
    responses = {}
    for model in models:
        prompt = f"Summarize this text:\n{text}"
        responses[model] = query_model(prompt, model=model)

    # Evaluate responses using a larger model
    evaluator_responses = {}
    for model, response in responses.items():
        eval_prompt = (
            f"Given the input text `{text[:500]}...` "  # Truncate input text for evaluation prompt
            f"and the summary `{response}`, "
            f"score the summary on a scale from 0 to 100, where 100 is the best summary."
        )
        score = query_model(eval_prompt, model=evaluator_model)
        evaluator_responses[model] = {"summary": response, "score": score}

    return evaluator_responses

In [15]:
# Test the pipeline
pdf_path = "Desktop/Advancements in Artificial Intelligence and Machine Learning.pdf"  # Path to your PDF document
text = upload_and_convert_pdf(pdf_path)

In [ ]:
text ="""

The Rise of Ancient Civilizations: Key Developments

The history of ancient civilizations reveals significant advancements that shaped human societies. From 3000 BCE to 500 CE, regions like Mesopotamia, Egypt, the Indus Valley, and China became cradles of culture, innovation, and governance.

In Mesopotamia, the Sumerians introduced cuneiform writing, enabling the recording of laws, trade, and stories. The Code of Hammurabi, written in Babylon, became one of the earliest known legal systems. Their advancements in mathematics and astronomy laid the foundation for future scientific endeavors.

Egypt, known for its monumental pyramids and temples, thrived along the Nile River. The development of hieroglyphics facilitated record-keeping and religious texts. Pharaohs like Ramses II and Cleopatra played crucial roles in shaping diplomacy and trade. The civilization's achievements in medicine, architecture, and irrigation were groundbreaking.

In the Indus Valley, cities like Mohenjo-Daro and Harappa showcased advanced urban planning, with grid layouts and sophisticated drainage systems. Their trade networks connected them to Mesopotamia and Central Asia, although their script remains undeciphered, leaving their societal structure a mystery.

China's Shang and Zhou dynasties contributed to advances in bronze work, silk production, and centralized administration. The Zhou's Mandate of Heaven introduced the concept of divine authority, influencing governance for centuries. Philosophical schools, including Confucianism and Daoism, emerged during this period, shaping ethics and societal values.

The Mediterranean also saw the rise of influential civilizations. Greece contributed significantly to philosophy, science, and democracy, with figures like Socrates, Aristotle, and Plato leading intellectual revolutions. Rome, succeeding Greece, built a vast empire known for its legal systems, engineering marvels like aqueducts, and enduring cultural contributions.

These ancient civilizations laid the groundwork for modern societies, with their innovations in governance, science, and culture continuing to inspire the present day. As we study their achievements, we gain insights into the enduring quest for progress and the challenges they overcame to build lasting legacies.


"""

In [19]:
print(text)
print(len(text))

Advancements in Artificial Intelligence and Machine Learning  
 
The field of Artificial Intelligence (AI) has seen significant advancements over the last few years. In 
2024, several leading institutions, including Harvard University, MIT, Stanford University, and The 
University of Oxford, have collaborated with tech gi ants like Microsoft, Google, OpenAI, and 
Amazon to push the boundaries of AI research.  
 
A key event was the Global AI Summit 2024, held in New York City in March, where prominent 
researchers like Dr. Elizabeth Green and Professor Alan Turing presented their findings on Quantum 
Computing and Neural Networks. The summit, organized by the United  Nations and World 
Economic Forum, was attended by over 5,000 professionals, including Elon Musk and Satya 
Nadella. Topics included ethical implications of AI, machine learning scalability, and the future of 
GPT-5 and LLaMA models.  
 
In April 2024, Tesla launched a new AI -powered autonomous vehicle model named Tesla M

In [16]:
# Smaller models to test
smaller_models = ["llama3.2:latest", "llama3.2:1b"]

# Evaluate summaries
results = evaluate_responses(text, smaller_models)

# Print results
for model, result in results.items():
    print(f"\nModel: {model}")
    print("Summary:")
    print(result["summary"])
    print("Score:")
    print(result["score"])
    print("\n-------------------------")


Model: llama3.2:latest
Summary:
The text discusses recent advancements in Artificial Intelligence (AI) and Machine Learning. Key developments include:

1. Collaboration between top institutions and tech giants like Microsoft, Google, and Amazon to push AI research boundaries.
2. The Global AI Summit 2024, where prominent researchers presented findings on Quantum Computing and Neural Networks.
3. Tesla's launch of an AI-powered autonomous vehicle model, incorporating Computer Vision and Natural Language Processing technologies developed in partnership with DeepMind.
4. Healthcare innovations using AI, such as tools for early disease detection by Pfizer, Roche, and Johnson & Johnson, and IBM Watson Health's collaboration with Mayo Clinic to enhance diagnostic capabilities.
5. Meta Platforms' updates to Reality Labs, focusing on Virtual Reality (VR) and Augmented Reality (AR) technologies.
6. NASA and SpaceX's collaboration on AI-driven navigation systems for deep space exploration missi

In [8]:
model = "llama3.2:latest"
result = query_model("What do Llamas eat", model)
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and grassy weeds.
2. Hay: High-quality hay, such as timothy hay or alfalfa hay, is a staple in a llama's diet. It provides essential nutrients like fiber, protein, and vitamins.
3. Grains: Llamas may also be fed grains like oats, barley, or corn, but these should not make up more than 10% of their diet.
4. Fruits and vegetables: Fresh fruits and vegetables, such as apples, carrots, and sweet potatoes, can be given to llamas as treats or added to their hay.
5. Browse: Llamas may also eat browse, which includes leaves, twigs, and other vegetation from trees and shrubs.

It's essential to note that llamas have a unique digestive system, with a four-chambered stomach, which allows them to break down and extract nutrients from plant material more efficiently than many other 